In [11]:
import pandas as pd
import random 
import matplotlib.pyplot as plt
from fuzzywuzzy import fuzz
from collections import defaultdict
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode
import openai
import ast
import sqlite3
from tqdm import tqdm
from urllib.parse import quote

# Load the file ENTPTNERDOI.csv.tar.gz from https://zenodo.org/records/10022727

In [ ]:
# Specify the path to the .tar.gz file
tar_gz_path = 'ENTPTNERDOI.csv.tar.gz'

# Specify the directory where you want to extract the file
extract_dir = 'extracted_files'

# Extract the tar.gz file
with tarfile.open(tar_gz_path, 'r:gz') as file:
    file.extractall(path=extract_dir)

In [39]:
ENTDOI = pd.read_csv('ENTPTNERDOI.csv')

In [3]:
ENTDOI.head()

,Entity,Part_of_text,NER_Tag,DOI,Preferred_Entity,Year
0,electrospray ionization mass spectrometry,Abs,APL,10.1016/s0925-4005(02)00161-2,Electrospray Ionization Mass Spectrometry,2002.0
1,wsoc,Abs,APL,10.1016/j.atmosenv.2007.06.034,WSOC,2007.0
2,pyridinium,Abs,APL,10.1021/jo0712401,Pyridinium,2007.0
3,navy blue,Abs,APL,10.1016/j.ecoleng.2013.07.005,Navy Blue,2013.0
4,gantt chart,Abs,APL,10.1021/ie970286p,Gantt Chart,1997.0


### Perform SQLite Query

Given the size of the ENTDOI.csv file, a pandas based aggregation and grouping can consume a lot of RAM \
Therefore, the query is performed using sqllite

In [4]:
# Assuming ENT_DF is your existing DataFrame
# Create a new SQLite database in memory
conn = sqlite3.connect(":memory:")

# Write ENT_DF to the SQLite database
ENTDOI.to_sql('ENT_DF', conn, index=False, if_exists='replace')

84659831

In [5]:
# SQL query to generate the new SRO_DF
sql_query = """
SELECT 
    E1.Preferred_Entity AS Subject, 
    E2.Preferred_Entity AS Object, 
    E1.NER_Tag || '-' || E2.NER_Tag AS Rel,
    COUNT(DISTINCT E1.DOI) AS Count
FROM 
    ENT_DF AS E1
JOIN 
    ENT_DF AS E2 ON E1.DOI = E2.DOI AND E1.Part_of_text = E2.Part_of_text
WHERE 
    E1.Preferred_Entity != E2.Preferred_Entity
GROUP BY 
    Subject, Object, Rel
HAVING 
    Count >= 30
"""

In [6]:
# Execute the SQL query and read the results into a new DataFrame
SRO_DF = pd.read_sql_query(sql_query, conn)
SRO_DF.reset_index(inplace = True, drop=True)

### Saving the results as SUBRELOBJ.csv

In [7]:
SRO_DF.to_csv('SUBRELOBJ.csv', index = None)

To ensure that the URIs for the preferred_entities are the same in ENTNERPRDOI and SUBRELOBJ,\
we save all the URIs to an entity_uri_dict and load the values into the SUBRELOBJ.nt

In [14]:
# Dictionary to keep track of entity URI mappings
entity_uri_mapping = {}
ner_dict           = {}
NER, pref_entity = ENTDOI[['NER_Tag', 'Preferred_Entity']].values.T
for ner, pref_ent in tqdm(zip(NER, pref_entity), total = len(NER)):
    if pref_ent:

        # Add Preferred_Entity, creating a URI based on NER tag
        preferred_entity_uri = URIRef("http://example.com/{}/{}".format(ner, quote(pref_ent)))

        entity_uri_mapping[pref_ent] = preferred_entity_uri
        ner_dict[pref_ent]           = ner

100%|██████████| 84659831/84659831 [02:57<00:00, 477442.16it/s]


In [23]:
import pickle
pickle.dump(entity_uri_mapping, open('entity_uri_mapping.pickle', 'wb'))

In [35]:
NERDF = pd.DataFrame.from_dict(ner_dict, orient = 'index')
NERDF.groupby(0).size().reset_index(name='count').sort_values('count', ascending = False)

,0,count
4,PRO,37265
1,CHM,31182
0,APL,27134
5,SMT,21027
2,CMT,16797
6,SPL,8656
3,DSC,6909


In [29]:
unique_entities = list(set(SRO_DF['Subject'].tolist() + SRO_DF['Object'].tolist()))
print("Number of unique entities : {}".format(len(unique_entities)))

Number of unique entities : 70002


In [31]:
reduced_ner_dict ={key: ner_dict[key] for key in unique_entities}
len(reduced_ner_dict)

70002

In [34]:
RNERDF = pd.DataFrame.from_dict(ner_dict, orient = 'index')
RNERDF.groupby(0).size().reset_index(name='count').sort_values('count', ascending = False)

,0,count
4,PRO,37265
1,CHM,31182
0,APL,27134
5,SMT,21027
2,CMT,16797
6,SPL,8656
3,DSC,6909


In [13]:
g = Graph()
# Assuming you have a DataFrame df with 'subject', 'object', 'relationship', 'count' columns
for row_num, row in tqdm(SRO_DF.iterrows(), total=len(SRO_DF)):
    subject, object_, relationship, count = row
    index = row_num  # Assuming row_num serves as the index

    # Get URIs from the dictionaries
    subject_uri = entity_uri_mapping.get(subject)
    object_uri = entity_uri_mapping.get(object_)
    relationship_uri = URIRef(f"http://example.org/{relationship}")

    # Create a blank node or URI for the index
    index_node = BNode(f"index{index}")

    # Add the triples to the graph
    g.add((index_node, URIRef("http://example.org/hasSubject"), URIRef(subject_uri)))
    g.add((index_node, URIRef("http://example.org/hasObject"), URIRef(object_uri)))
    g.add((index_node, URIRef("http://example.org/hasRelationship"), URIRef(relationship_uri)))
    g.add((index_node, URIRef("http://example.org/hasCount"), Literal(count)))
    
g.serialize(destination='SUBRELOBJ.nt', format='nt')  

100%|██████████| 5411478/5411478 [09:58<00:00, 9042.54it/s]  
/home/synthesisproject/anaconda3/envs/vineeth_10/lib/python3.10/site-packages/rdflib/plugins/serializers/nt.py:40: UserWarning: NTSerializer always uses UTF-8 encoding. Given encoding was: None
  warnings.warn(


<Graph identifier=Nc8fcafd346764f54a963323d12bd0f4b (<class 'rdflib.graph.Graph'>)>